# Imports

In [1]:
from pathlib import Path
import requests
import os
from dotenv import load_dotenv
import pandas as pd

ENV_PATH = Path().resolve().parent.parent / ".env"
load_dotenv(dotenv_path=ENV_PATH)
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
if not OPENROUTER_API_KEY:
    raise RuntimeError(f"OPENROUTER_API_KEY not found in {ENV_PATH}")

headers = {"Authorization": f"Bearer {OPENROUTER_API_KEY}"}

## Expenditures

In [3]:
def get_usage_data(api_url, api_key):
    headers = {
        "Authorization": f"Bearer {api_key}"
    }
    response = requests.get(api_url, headers=headers)
    return response.json()["data"]["usage"]

usage = get_usage_data(api_url='https://openrouter.ai/api/v1/auth/key', api_key=OPENROUTER_API_KEY)
print(f'Потрачено на текущий момент: {usage} USD')

Потрачено на текущий момент: 62.058554475 USD


## Available models

In [8]:
df_models_OR = pd.json_normalize(requests.get('https://openrouter.ai/api/v1/models', headers=headers).json()['data'])
df_models_OR["created_dt"] = pd.to_datetime(df_models_OR["created"], unit="s")
df_models_OR.sort_values(by='created_dt', ascending=False).head() # Top 5 latest

,id,hugging_face_id,name,created,description,context_length,per_request_limits,supported_parameters,architecture.modality,architecture.input_modalities,...,pricing.request,pricing.image,pricing.web_search,pricing.internal_reasoning,top_provider.context_length,top_provider.max_completion_tokens,top_provider.is_moderated,pricing.input_cache_read,pricing.input_cache_write,created_dt
0,meta-llama/llama-3.3-8b-instruct:free,,Meta: Llama 3.3 8B Instruct (free),1747230154,A lightweight and ultra-fast variant of Llama ...,12800,None,"[max_tokens, temperature, top_p, structured_ou...",text->text,[text],...,0,0,0,0,12800.0,4028.0,False,NaN,NaN,2025-05-14 13:42:34
1,nousresearch/deephermes-3-mistral-24b-preview:...,NousResearch/DeepHermes-3-Mistral-24B-Preview,Nous: DeepHermes 3 Mistral 24B Preview (free),1746830904,DeepHermes 3 (Mistral 24B Preview) is an instr...,32768,None,"[max_tokens, temperature, top_p, reasoning, in...",text->text,[text],...,0,0,0,0,32768.0,NaN,False,NaN,NaN,2025-05-09 22:48:24
2,mistralai/mistral-medium-3,,Mistral: Mistral Medium 3,1746627341,Mistral Medium 3 is a high-performance enterpr...,131072,None,"[tools, tool_choice, max_tokens, temperature, ...",text+image->text,"[text, image]",...,0,0,0,0,131072.0,NaN,False,NaN,NaN,2025-05-07 14:15:41
3,google/gemini-2.5-pro-preview,,Google: Gemini 2.5 Pro Preview,1746578513,Gemini 2.5 Pro is Google’s state-of-the-art AI...,1048576,None,"[max_tokens, temperature, top_p, tools, tool_c...",text+image->text,"[text, image, file]",...,0,0.00516,0,0,1048576.0,65535.0,False,0.00000031,0.000001625,2025-05-07 00:41:53
4,arcee-ai/caller-large,,Arcee AI: Caller Large,1746487869,"Caller Large is Arcee's specialist ""function‑c...",32768,None,"[tools, tool_choice, max_tokens, temperature, ...",text->text,[text],...,0,0,0,0,32768.0,NaN,False,NaN,NaN,2025-05-05 23:31:09


### Google nultimodal models (`text+image->text`) for description generation

In [9]:
df_models_OR[
    (df_models_OR['architecture.modality'] == 'text+image->text') &
    (df_models_OR['id'].str.startswith('google')) &
    (~df_models_OR['id'].str.endswith('free'))
].sort_values(by='pricing.prompt', ascending=True).reset_index(drop=True)

,id,hugging_face_id,name,created,description,context_length,per_request_limits,supported_parameters,architecture.modality,architecture.input_modalities,...,pricing.request,pricing.image,pricing.web_search,pricing.internal_reasoning,top_provider.context_length,top_provider.max_completion_tokens,top_provider.is_moderated,pricing.input_cache_read,pricing.input_cache_write,created_dt
0,google/gemini-2.5-pro-exp-03-25,,Google: Gemini 2.5 Pro Experimental,1742922099,Gemini 2.5 Pro is Google’s state-of-the-art AI...,1048576,None,"[max_tokens, temperature, top_p, tools, tool_c...",text+image->text,"[text, image, file]",...,0,0,0,0,1048576.0,65536.0,False,NaN,NaN,2025-03-25 17:01:39
1,google/gemma-3-4b-it,google/gemma-3-4b-it,Google: Gemma 3 4B,1741905510,"Gemma 3 introduces multimodality, supporting v...",131072,None,"[max_tokens, temperature, top_p, stop, frequen...",text+image->text,"[text, image]",...,0,0,0,0,131072.0,NaN,False,NaN,NaN,2025-03-13 22:38:30
2,google/gemini-flash-1.5-8b,None,Google: Gemini 1.5 Flash 8B,1727913600,Gemini Flash 1.5 8B is optimized for speed and...,1000000,None,"[max_tokens, temperature, top_p, stop, frequen...",text+image->text,"[text, image]",...,0,0,0,0,1000000.0,8192.0,False,0.00000001,0.0000000583,2024-10-03 00:00:00
3,google/gemma-3-12b-it,google/gemma-3-12b-it,Google: Gemma 3 12B,1741902625,"Gemma 3 introduces multimodality, supporting v...",131072,None,"[max_tokens, temperature, top_p, stop, frequen...",text+image->text,"[text, image]",...,0,0,0,0,131072.0,NaN,False,NaN,NaN,2025-03-13 21:50:25
4,google/gemini-2.0-flash-lite-001,,Google: Gemini 2.0 Flash Lite,1740506212,Gemini 2.0 Flash Lite offers a significantly f...,1048576,None,"[tools, tool_choice, max_tokens, temperature, ...",text+image->text,"[text, image, file]",...,0,0,0,0,1048576.0,8192.0,False,NaN,NaN,2025-02-25 17:56:52
5,google/gemini-flash-1.5,None,Google: Gemini 1.5 Flash,1715644800,Gemini 1.5 Flash is a foundation model that pe...,1000000,None,"[max_tokens, temperature, top_p, stop, frequen...",text+image->text,"[text, image]",...,0,0.00004,0,0,1000000.0,8192.0,False,0.00000001875,0.0000001583,2024-05-14 00:00:00
6,google/gemma-3-27b-it,,Google: Gemma 3 27B,1741756359,"Gemma 3 introduces multimodality, supporting v...",131072,None,"[max_tokens, temperature, top_p, stop, frequen...",text+image->text,"[text, image]",...,0,0.0000256,0,0,131072.0,16384.0,False,NaN,NaN,2025-03-12 05:12:39
7,google/gemini-2.0-flash-001,,Google: Gemini 2.0 Flash,1738769413,Gemini Flash 2.0 offers a significantly faster...,1048576,None,"[tools, tool_choice, max_tokens, temperature, ...",text+image->text,"[text, image, file]",...,0,0.0000258,0,0,1048576.0,8192.0,False,0.000000025,0.0000001833,2025-02-05 15:30:13
8,google/gemini-2.5-flash-preview,,Google: Gemini 2.5 Flash Preview,1744914667,Gemini 2.5 Flash is Google's state-of-the-art ...,1048576,None,"[max_tokens, temperature, top_p, tools, tool_c...",text+image->text,"[image, text, file]",...,0,0.0006192,0,0,1048576.0,65535.0,False,0.0000000375,0.0000002333,2025-04-17 18:31:07
9,google/gemini-2.5-flash-preview:thinking,,Google: Gemini 2.5 Flash Preview (thinking),1744914667,Gemini 2.5 Flash is Google's state-of-the-art ...,1048576,None,"[max_tokens, temperature, top_p, tools, tool_c...",text+image->text,"[image, text, file]",...,0,0.0006192,0,0,1048576.0,65535.0,False,0.0000000375,0.0000002333,2025-04-17 18:31:07
